In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 30)
types = {'id': int, 'ciudad':str, 'provincia': str, 'precio': float}

training_data = pd.read_csv('../../data/train.csv', dtype = types, usecols=['id','provincia','ciudad','precio'])
evaluation_data = pd.read_csv('../../data/test.csv',dtype = types, usecols=['id','provincia','ciudad'])
training_data.head()

,id,ciudad,provincia,precio
0,254099,Benito Juárez,Distrito Federal,2273000.0
1,53461,La Magdalena Contreras,Distrito Federal,3600000.0
2,247984,Tonalá,Jalisco,1200000.0
3,209067,Zinacantepec,Edo. de México,650000.0
4,185997,Zapopan,Jalisco,1150000.0


In [2]:
training_data['ciudad'].fillna('sinciudad', inplace = True)
training_data['provincia'].fillna('sinprovincia', inplace = True)
evaluation_data['ciudad'].fillna('sinciudad', inplace = True)
evaluation_data['provincia'].fillna('sinprovincia', inplace = True)

In [3]:
training_data['provincia'] = training_data['provincia'].apply(lambda x: x.lower())
training_data['ciudad'] = training_data['ciudad'].apply(lambda x: x.lower())
evaluation_data['provincia'] = evaluation_data['provincia'].apply(lambda x: x.lower())
evaluation_data['ciudad'] = evaluation_data['ciudad'].apply(lambda x: x.lower())

In [4]:
clasificacion_de_aglomeracion = {
                    "cajeme":"ciudad_muy_pequeña",
                    "victoria":"ciudad_muy_pequeña",
                    "coatzacoalcos":"ciudad_muy_pequeña",
                    "colima":"ciudad_muy_pequeña", 
                    "córdoba":"ciudad_muy_pequeña",
                    "cuautla":"ciudad_muy_pequeña",
                    "ensenada":"ciudad_muy_pequeña",
                    "irapuato":"ciudad_muy_pequeña",
                    "mazatlán":"ciudad_muy_pequeña",
                    "minatitlán":"ciudad_muy_pequeña",
                    "monclova":"ciudad_muy_pequeña",
                    "nuevo laredo":"ciudad_muy_pequeña",
                    "orizaba":"ciudad_muy_pequeña",
                    "puerto vallarta":"ciudad_muy_pequeña",
                    "tehuacán":"ciudad_muy_pequeña",
                    "tepic":"ciudad_muy_pequeña",
                    "zacatecas":"ciudad_muy_pequeña",                   
                    "acapulco de juárez":"ciudad_pequeña",
                    "cancún":"ciudad_pequeña",
                    "celaya":"ciudad_pequeña",
                    "chihuahua":"ciudad_pequeña",
                    "cuernavaca":"ciudad_pequeña",
                    "culiacán":"ciudad_pequeña",
                    "durango":"ciudad_pequeña",
                    "hermosillo":"ciudad_pequeña",
                    "matamoros":"ciudad_pequeña",
                    "morelia":"ciudad_pequeña",
                    "oaxaca de juárez":"ciudad_pequeña",
                    "pachuca":"ciudad_pequeña",
                    "poza rica de hidalgo":"ciudad_pequeña",
                    "reynosa":"ciudad_pequeña",
                    "saltillo":"ciudad_pequeña",
                    "tampico":"ciudad_pequeña",
                    "tlaxcala":"ciudad_pequeña",
                    "tuxtla gutiérrez":"ciudad_pequeña",
                    "veracruz":"ciudad_pequeña",
                    "villahermosa":"ciudad_pequeña",
                    "xalapa":"ciudad_pequeña",
                    "aguascalientes":"ciudad_mediana",
                    "juárez":"ciudad_mediana",
                    "guadalajara":"ciudad_mediana",
                    "león":"ciudad_mediana",
                    "mérida":"ciudad_mediana",
                    "mexicali":"ciudad_mediana",
                    "monterrey":"ciudad_mediana",
                    "puebla":"ciudad_mediana",
                    "querétaro":"ciudad_mediana",
                    "san luis potosí":"ciudad_mediana",
                    "tijuana":"ciudad_mediana",
                    "toluca":"ciudad_mediana"
                   }

In [5]:
def tipo_de_aglomeracion(data):
    ciudad = data[1]
    if data[2] == "distrito federal": return "megaciudad"
    if ciudad in clasificacion_de_aglomeracion: return clasificacion_de_aglomeracion[ciudad]
    else: return "ciudad_rural"

In [6]:
training_data['tipo_de_aglomeracion'] = training_data.apply(tipo_de_aglomeracion, axis=1)
evaluation_data['tipo_de_aglomeracion'] = training_data.apply(tipo_de_aglomeracion, axis=1)

In [7]:
mean_precios = training_data.groupby('tipo_de_aglomeracion').agg({'precio':'mean'}).reset_index()

precio_mean_by_type_of_agglomeration = {}
for tipo in mean_precios['tipo_de_aglomeracion'].unique():
    precio_mean_by_type_of_agglomeration[tipo] = mean_precios.loc[mean_precios['tipo_de_aglomeracion']==tipo]['precio'].values[0]

In [8]:
training_data['mean_precio_encoded_type_of_agglomeration'] = training_data['tipo_de_aglomeracion'].apply(lambda x: precio_mean_by_type_of_agglomeration[x])
evaluation_data['mean_precio_encoded_type_of_agglomeration'] = evaluation_data['tipo_de_aglomeracion'].apply(lambda x: precio_mean_by_type_of_agglomeration[x])

training_data.drop(columns=['tipo_de_aglomeracion','ciudad','provincia', 'precio'], inplace = True)
evaluation_data.drop(columns=['tipo_de_aglomeracion','ciudad','provincia'], inplace = True)

evaluation_data.head()

,id,mean_precio_encoded_type_of_agglomeration
0,4941,3.471705e+06
1,51775,3.471705e+06
2,115253,2.410765e+06
3,299321,2.410765e+06
4,173570,2.410765e+06


In [9]:
training_data.head()

,id,mean_precio_encoded_type_of_agglomeration
0,254099,3.471705e+06
1,53461,3.471705e+06
2,247984,2.410765e+06
3,209067,2.410765e+06
4,185997,2.410765e+06


In [11]:
evaluation_data.to_csv('../../res/ftr/mean_precio_encoded_type_of_agglomeration_evaluation.csv')
training_data.to_csv('../../res/ftr/mean_precio_encoded_type_of_agglomeration_train.csv')